In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
path = "/content/drive/MyDrive/TrueCars/df_fullR.csv"
df_full = pd.read_csv(path)
df_full.sample(5)

,Price,Year,Mileage,City,State,Make,Model,Usage_level,City_importance
340673,11700,2008,127735,SHORELINE,WA,TOYOTA,RAV44WD,High,Low
996835,12500,2016,36377,GILROY,CA,NISSAN,ALTIMA2.5,Medium,Low
154949,41950,2016,24616,CARROLLTON,TX,BMW,5,Low,Low
856666,6999,2006,130476,CUMMING,GA,JEEP,COMMANDER4DR,High,Low
427392,31000,2017,7971,MIAMI,FL,CADILLAC,XTSFWD,Low,Low


In [10]:
df_full = df_full[df_full.City_importance != "Low"]
df_full.drop(columns=['City','State','City_importance'],inplace=True)
df_full.sample(5)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Price,Year,Mileage,Make,Model,Usage_level
109966,28790,2015,24481,CHEVROLET,SILVERADO,Low
1196182,17995,2015,35661,VOLKSWAGEN,TIGUANSEL,Medium
223343,14746,2015,59895,KIA,SORENTOLX,Medium
1170742,28910,2014,57544,TOYOTA,4RUNNERSR5,Medium
347489,10000,2014,30271,VOLKSWAGEN,JETTA,Medium


In [11]:
X = df_full.iloc[:,1:]
y = df_full.iloc[:,0]

In [12]:
X.head()

,Year,Mileage,Make,Model,Usage_level
9,2015,13807,BUICK,ENCORECONVENIENCE,Low
10,2014,29535,BUICK,ENCOREFWD,Medium
18,2015,20681,BUICK,ENCORECONVENIENCE,Low
24,2015,21694,BUICK,ENCOREFWD,Low
26,2014,12760,BUICK,ENCOREFWD,Low


In [13]:
y.head()

9     17700
10    14555
18    15239
24    16400
26    16450
Name: Price, dtype: int64

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y ,test_size=0.1,random_state=2)

In [15]:
from sklearn.ensemble import GradientBoostingRegressor

In [16]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [17]:
ohe=OneHotEncoder()
ohe.fit(X[['Make','Model','Usage_level']])

OneHotEncoder()

In [18]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Make','Model','Usage_level']),
                                    remainder='passthrough')

In [19]:
gbr_params = {'n_estimators': 500,
          'max_depth': 3,
          'min_samples_split': 5,
          'learning_rate': 1,
          'loss': 'squared_error'}

In [20]:
gbr = GradientBoostingRegressor(**gbr_params)

In [21]:
pipe=make_pipeline(column_trans,gbr)

In [22]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['ACURA', 'ALFA', 'AM', 'ASTON', 'AUDI', 'BENTLEY', 'BMW', 'BUICK',
       'CADILLAC', 'CHEVROLET', 'CHRYSLER', 'DODGE', 'FERRARI', 'FIAT',
       'FISKER', 'FORD', 'FREIGHTLINER', 'GENESIS', 'GMC', 'HONDA',
       'HUMMER', 'HYUNDAI', 'INFINITI', 'ISU...
       'PORSCHE', 'RAM', 'ROLLS-ROYCE', 'SAAB', 'SATURN', 'SCION',
       'SMART', 'SUBARU', 'SUZUKI', 'TESLA', 'TOYOTA', 'VOLKSWAGEN',
       'VOLVO'], dtype=object),
                                                                            array(['1', '124', '15002WD', ..., 'ZDXAWD', 'ZEPHYR4DR', 'ZX22DR'],
      dtype=object),
                                                                            array(['High', 'Low', 'Medium'], dtype=object)]),
                                     

In [23]:
df_full.sample(5)

,Price,Year,Mileage,Make,Model,Usage_level
414720,4495,2003,100280,BUICK,RENDEZVOUSCX,High
995761,15500,2016,39978,MITSUBISHI,LANCERES,Medium
972747,27995,2014,48381,MERCEDES-BENZ,E-CLASSE350,Medium
222188,14988,2016,3474,KIA,SOULAUTOMATIC,Low
264823,12999,2015,14808,NISSAN,ALTIMA2.5,Low


In [24]:
y_pred=pipe.predict(X_test)

In [25]:
r2_score(y_test,y_pred)

0.9013744800927942

In [26]:
pipe.predict(pd.DataFrame(columns=['Year','Make','Model','Mileage','Usage_level'],data=np.array([7, 'KIA', 'OPTIMALX', 35279, 'Medium']).reshape(1,5)))

array([62227.68963593])

In [27]:
pipe.predict(pd.DataFrame(columns=['Year','Make','Model','Mileage','Usage_level'],data=np.array([7, 'TOYOTA', 'TACOMA4WD', 54258, 'Medium']).reshape(1,5)))

array([44293.77056961])

In [28]:
import pickle

In [29]:
pickle.dump(pipe,open('GBR.pkl','wb'))